# MATLAB => XMILE

In [14]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
import re
import xml.etree.ElementTree as etree

from py_expression_eval import Parser
parser = Parser()

In [15]:
source_xmlns = "{http://docs.oasis-open.org/xmile/ns/XMILE/v1.0}" 
source_xmlns_isee = "{http://iseesystems.com/XMILE}"

In [16]:
matlab_file = 'goodwin-minsky-matlab/goodwin-minsky-with-names.m'
template_xmile = 'template-xmile.xml'

# Parseo archivo Matlab

In [17]:
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

In [18]:
with open(matlab_file, 'rb') as f:
    matlab_code = f.read()
    #print(matlab_code)

In [19]:
# stocks ids
stocks_ids_re = re.compile(r"[\w]+=x\(\d+\)")
stocks_ids = {re.search(re.compile(r"(.+)="), exp).group(0)[:-1] : 
              re.search(re.compile(r"x\((.+)\)"), exp).group(1)
              for exp in stocks_ids_re.findall(matlab_code)}
#print(stocks_ids)

stocks_init_values_re = re.compile(r"x0\(\d+\)=\d+[\.\d+]*")
stocks_init_values_dict = {re.search(re.compile(r"x0\((.+)\)"), exp).group(1) : 
                    re.search(re.compile(r"=(.+)"), exp).group(0)[1:]
                    for exp in stocks_init_values_re.findall(matlab_code)}
#print(stocks_init_values_dict)

### Joineo los 'stocks' con sus valores iniciales Capital=x(1) => x0(1)=300

In [20]:
stocks_init_values_re = re.compile(r"[\w]+=x\(\d+\)")
stocks_init_values = {re.search(re.compile(r"(.+)="), exp).group(0)[:-1] : 
                    stocks_init_values_dict[re.search(re.compile(r"x\((.+)\)"), exp).group(1)]
                    for exp in stocks_init_values_re.findall(matlab_code)}
#print(stocks_init_values)

In [21]:
stocks_change_re = re.compile(r"f\(\d+\)=[\w\(\)\+\-\*\/\.]+")
stocks_change_dict = {re.search(re.compile(r"f\((\d+)\)"), exp).group(1) : 
                      parser.parse(re.search(re.compile(r"=(.+)"), exp).group(0)[1:])
                      for exp in stocks_change_re.findall(matlab_code)}
#print(stocks_change_dict)

### Joineo los 'stocks' con sus rates de cambio Capital=x(1) => f(1)=InvestmentNetReal

In [22]:
stocks_change = {k : stocks_change_dict[stocks_ids[k]]
                 for k,v in stocks_init_values.iteritems()}
#print({k : v.toString() for k,v in stocks_change.iteritems()})

In [23]:
auxs_init_values_re = re.compile(r"[\w]+=\d+[\.\d+]*")
auxs_init_values = {re.search(re.compile(r"(.+)="), exp).group(0)[:-1] : 
                    re.search(re.compile(r"=(.+)"), exp).group(0)[1:] 
                    for exp in auxs_init_values_re.findall(matlab_code)}
#print(auxs_init_values)

In [24]:
# TODO : solo acepta operaciones de +, -, *, /
auxs_equations_re = re.compile(r"[\w]+=[\w\(\)\+\-\*\/\.]+")
auxs_equations = {re.search(re.compile(r"(.+)="), exp).group(0)[:-1] : 
                    parser.parse(re.search(re.compile(r"=(.+)"), exp).group(0)[1:])
                    for exp in auxs_equations_re.findall(matlab_code)[1:]
                    if re.search(re.compile(r"(.+)="), exp).group(0)[:-1] not in
                    auxs_init_values.keys() + stocks_init_values.keys()}
#print(auxs_equations.keys())
#print([exp.simplify({}).toString() for exp in auxs_equations.values()])

## Estructura de datos con informacion de stocks / flows / auxs

In [25]:
# stocks_init_values | auxs_init_values | flows_auxs_equation
stocks = [
    {'name' : k, 'eqn' : v, 'inflow' : 'chg'+k} 
    for k,v in stocks_init_values.iteritems()
]
flows = [
    {'name' : 'chg'+k, 'eqn' : stocks_change[k].toString()}
    for k,v in stocks_init_values.iteritems()
]
# Auxs : definidas de forma DIRECTA / INDIRECTA
auxs = [
    {'name' : k, 'eqn' : v.toString()}
    for k,v in auxs_equations.iteritems()
] + [
    {'name' : k, 'eqn' : v}
    for k,v in auxs_init_values.iteritems()
]
dependencies = [
    {'name' : 'chg'+k, 'inputs' : stocks_change[k].variables()}
    for k,v in stocks_init_values.iteritems()
] + [
    {'name' : k, 'inputs' : v.variables()}
    for k,v in auxs_equations.iteritems()
]

In [26]:
#stocks

In [27]:
#flows

In [28]:
#auxs

In [29]:
#dependencies

# Generacion de archivo XMILE

In [30]:
model_name = 'test'
time_start = 1
time_stop = 2
time_delta = 0.5

In [31]:
context = {
    'model_name' : model_name,
    'time_start' : time_start,
    'time_stop'  : time_stop,
    'time_delta' : time_delta, 
    'stocks' : stocks,
    'flows'  : flows,
    'auxs'   : auxs,
    'dependencies' : dependencies
}
#print(render_template(template_xmile, context))

In [19]:
with open('goodwin-minsky-matlab/goodwin-minsky-matlab.xmile', 'w') as f:
    f.write(render_template(template_xmile, context))